# Cleaning Melbourne sensor footfall data 
This script cleans the footfall data for Melbourne

#### Code initialisation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from datetime import date, datetime

#### Import pedestrian count data

In [13]:
sensor_counts = pd.read_csv('/nfs/a319/gy17m2a/Project/footfall/footfall/FootfallScripts/MelbourneAnalysis/Data/Data/FootfallData/Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022.csv')
sensor_locations = pd.read_csv('../../Data/Data/FootfallData/melbourne_locations.csv')

In [14]:
len(sensor_counts['sensor_id'].unique())

82

#### Rename columns to all lowercase (to facilitate joining)

In [15]:
sensor_counts.rename({'Date_Time': 'datetime', 'Year': 'year', 'Month':'month', 'Mdate': 'mdate', 
                      'Day': 'day', 'Time': 'time', 'Sensor_ID': 'sensor_id', 'Hourly_Counts': 'hourly_counts'}, 
                     axis = 1, inplace = True)

#### Drop unneeded columns

In [17]:
# sensor_counts.drop(['sensor_id'], axis = 1, inplace = True)
sensor_locations.drop(['sensor_name', 'installation_date', 'status', 'note', 'direction_1',
                      'direction_2'], axis = 1, inplace = True)

#### Join two dataframes so location and count info in same place

In [18]:
location_counts = pd.merge(sensor_locations, sensor_counts, on='sensor_id', how='inner')

#### It could be useful to drop sensors that do not have as many recorded countsfor now leaving them in, but may drop in the future

#### Properly format datetime column

In [19]:
location_counts['datetime'] = pd.to_datetime(location_counts['datetime'], format = '%B %d, %Y %I:%M:%S %p')

#### Order by datetime column

In [20]:
location_counts = location_counts.sort_values(by=['datetime'])
location_counts.reset_index(inplace = True, drop = True)

#### September, 2010 dates have problems:
All dates have a timestamp of 00:00, presume they are in order of hour of day, for each day there are only 23 hours worth of data
For now, in later stages just filter out 2010 data

In [21]:
location_counts = location_counts[location_counts['year']>2010]

### Clean and save data

In [22]:
# location_counts.drop(['Latitude', 'Longitude', 'location'], axis = 1, inplace =True)
# location_counts.rename({'day': 'weekday', 'mdate': 'day', 'time': 'hour'}, axis =1, inplace = True)

# Change month to integers 1-12
months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June':6, 'July': 7, 'August': 8,
         'September': 9, 'October': 10, 'November': 11, 'December': 12}

location_counts.month = location_counts.month.map(months)

In [23]:
just_some_sensors = location_counts[location_counts['sensor_id'].isin([2,6,8,9,10,11,18])]
# just_some_sensors.to_csv("../../Cleaned_data/validsensors.csv",index=False)
# location_counts.to_csv("../../Cleaned_data/allsensors.csv",index=False)

In [27]:
# Get data for just one sensor
one_sensor =  location_counts[location_counts.sensor_id == 1]
one_year = one_sensor[one_sensor.year==2010]
# Set the datetime as the index
one_sensor.set_index('datetime', inplace = True)
duplicates = one_sensor[one_sensor.index.duplicated()]
for day in duplicates.mdate.unique():
    one_day = duplicates[duplicates.mdate == day]
    print(day, len(one_day))

In [28]:
one_sensor

,sensor_id,Name,Latitude,Longitude,location,Start,End,Values_per_sensor,Cluster_num_5clusters,ID,year,month,mdate,day,time,Sensor_Name,hourly_counts
datetime,,,,,,,,,,,,,,,,,
2011-12-20 00:00:00,1,Bourke Street Mall (North),-37.813494,144.965153,"(-37.8134944, 144.96515324)",2011.0,2022.0,92659.0,1,398257,2011,12,20,Tuesday,0,Bourke Street Mall (North),83
2011-12-20 01:00:00,1,Bourke Street Mall (North),-37.813494,144.965153,"(-37.8134944, 144.96515324)",2011.0,2022.0,92659.0,1,398275,2011,12,20,Tuesday,1,Bourke Street Mall (North),30
2011-12-20 02:00:00,1,Bourke Street Mall (North),-37.813494,144.965153,"(-37.8134944, 144.96515324)",2011.0,2022.0,92659.0,1,398293,2011,12,20,Tuesday,2,Bourke Street Mall (North),28
2011-12-20 03:00:00,1,Bourke Street Mall (North),-37.813494,144.965153,"(-37.8134944, 144.96515324)",2011.0,2022.0,92659.0,1,398311,2011,12,20,Tuesday,3,Bourke Street Mall (North),11
2011-12-20 04:00:00,1,Bourke Street Mall (North),-37.813494,144.965153,"(-37.8134944, 144.96515324)",2011.0,2022.0,92659.0,1,398329,2011,12,20,Tuesday,4,Bourke Street Mall (North),12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31 19:00:00,1,Bourke Street Mall (North),-37.813494,144.965153,"(-37.8134944, 144.96515324)",2011.0,2022.0,92659.0,1,4522024,2022,10,31,Monday,19,Bourke Street Mall (North),978
2022-10-31 20:00:00,1,Bourke Street Mall (North),-37.813494,144.965153,"(-37.8134944, 144.96515324)",2011.0,2022.0,92659.0,1,4522025,2022,10,31,Monday,20,Bourke Street Mall (North),428
2022-10-31 21:00:00,1,Bourke Street Mall (North),-37.813494,144.965153,"(-37.8134944, 144.96515324)",2011.0,2022.0,92659.0,1,4522026,2022,10,31,Monday,21,Bourke Street Mall (North),338
